# PREPARATIVOS

In [1]:
import pandas as pd
import numpy as np
import itertools as it
import re
import os
from reconocimiento_ch import *
from matches_otras import *

In [2]:
%load_ext line_profiler

## MODIFICAR LA SOCIEDAD

In [3]:
# MODIFICAR AQUÏ
soc = 2200

## Cargar datos

In [4]:
%store -r ARCHIVOS KNA1

In [5]:
#sociedades = (2000, 3000, 3100)
#ARCHIVOS = {sociedad: {} for sociedad in sociedades}

In [6]:
def leer_de_sociedad(soc, ARCHIVOS):
    ARCHIVOS[soc]["FBL3N"] = leer_excel("{}/Soc {}-Bancos SAP.xlsx".format(soc, soc))
    ARCHIVOS[soc]["FBL5N"] = leer_excel("{}/Soc {}-PAs SAP.xlsx".format(soc, soc))

In [7]:
#leer_de_sociedad(soc, ARCHIVOS)

In [8]:
#KNA1 = leer_excel("KNA1.xlsx")

In [9]:
%store ARCHIVOS KNA1

Stored 'ARCHIVOS' (dict)
Stored 'KNA1' (DataFrame)


In [10]:
FBL3N = ARCHIVOS[soc]["FBL3N"]
FBL5N = ARCHIVOS[soc]["FBL5N"]

# PARTE 1

In [11]:
textos = FBL3N["Texto"]
FBL3N["RUT"] = textos.apply(detectar_rut)
FBL3N["N° Factura"] = textos.apply(detectar_n_factura)

RUT_FBL3N = FBL3N[pd.notna(FBL3N["RUT"])]
FACT_FBL3N = FBL3N[pd.isna(FBL3N["RUT"]) & pd.notna(FBL3N["N° Factura"])]

# PARTE 2

### Trabajar con calces de factura

In [12]:
FBL5N["Referencia"] = FBL5N["Referencia"].apply(lambda s: str(s))
FBL5N["Cuenta"] = FBL5N["Cuenta"][pd.notna(FBL5N["Cuenta"])].apply(lambda n: str(int(n)))

KNA1["Nº ident.fis.1"] = KNA1["Nº ident.fis.1"].apply(lambda s: str(s).lstrip("0"))
KNA1["Cliente"] = KNA1["Cliente"][pd.notna(KNA1["Cliente"])].apply(lambda s: str(int(s)))

In [13]:
# Calzar números de factura
filtro_fact = FBL5N["Referencia"].isin(FACT_FBL3N["N° Factura"])
FACT_FBL5N = FBL5N[filtro_fact]
NO_FACT_FBL5N = FBL5N[~filtro_fact]

In [14]:
# Calzar RUTs
RUT_ID_FBL3N = pd.merge(RUT_FBL3N, KNA1, left_on="RUT", right_on="Nº ident.fis.1")

filtro_id5 = NO_FACT_FBL5N["Cuenta"].isin(RUT_ID_FBL3N["Cliente"])
ID_FBL5N = NO_FACT_FBL5N[filtro_id5]
NN_FBL5N = NO_FACT_FBL5N[~filtro_id5]

filtro_id3 = RUT_ID_FBL3N["Cliente"].isin(ID_FBL5N["Cuenta"])
ID_FBL3N = RUT_ID_FBL3N[filtro_id3]
SOLO_RUT_FBL3N = RUT_ID_FBL3N[filtro_id3]

In [15]:
#MATCHES_FACT = pd.merge(FACT_FBL3N, FACT_FBL5N, left_on="N° Factura", right_on="Referencia")
#escribir_excel(FACT_FBL5N, "RESULTADOS/Facturas.xlsx")

# PARTE 3

## Matches

In [16]:
DF = calzar(ID_FBL3N, ID_FBL5N)

In [17]:
DF

,Grupo,Sociedad,Nº documento FBL3N,Clase de documento FBL3N,Importe en moneda local,Moneda local,Texto,Cliente,Nº ident.fis.1,Nombre 1,Asignación,Nº documento FBL5N,Referencia,Clase de documento FBL5N,Importe en moneda doc.,Moneda del documento


In [18]:
d = extraer_datos(ID_FBL3N, ID_FBL5N)
match_1_1(d)
match_1_T(d)
match_1_V(d)
#%lprun -f match_1_M match_1_M(d)

In [19]:
escribir_excel(DF, "{}/RUT y Monto {}.xlsx".format(soc, soc))

Escribiendo nuevo archivo Excel: 2200/RUT y Monto 2200.xlsx
Archivo Excel escrito.


# PARTE 4

### Determinar qué datos encontré yo que Roberto no, y viceversa

1. RUT y Monto

In [20]:
excel = leer_excel("{}/Soc {}-Propuesta SAP.xlsx".format(soc, soc), None)

Leyendo archivo Excel: 2200/Soc 2200-Propuesta SAP.xlsx
Archivo Excel leído.


In [21]:
try:
    RUT_Y_MONTO = excel["Rut y Monto"]
except:
    hoja_0 = next(iter(excel))
    RUT_Y_MONTO = pd.DataFrame(columns=excel[hoja_0].columns)

In [22]:
outer_rym = pd.merge(DF, RUT_Y_MONTO, left_on="Nº documento FBL3N", right_on="Nº documento", how="outer", indicator=True)

In [23]:
escribir_excel(outer_rym[outer_rym["_merge"] == "left_only"], "{}/RUT y Monto {} - LEFT.xlsx".format(soc, soc))

Escribiendo nuevo archivo Excel: 2200/RUT y Monto 2200 - LEFT.xlsx
Archivo Excel escrito.


In [24]:
escribir_excel(outer_rym[outer_rym["_merge"] == "right_only"], "{}/RUT y Monto {} - RIGHT.xlsx".format(soc, soc))

Escribiendo nuevo archivo Excel: 2200/RUT y Monto 2200 - RIGHT.xlsx
Archivo Excel escrito.
